**Approach used:** SARIMAX (Seasonal Autoregressive Integrated Moving Average with eXogeneous variables)

**Reason:** The data provided is seasonal, and it is a time series data with multiple exogeneous variables influencing the result. Hence, the optimal statistical model that can be applied to this task is SARIMAX

**Main Modules Used:**

statsmodel package in Python

## Import Required Modules

In [4]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import statsmodels.api as sm
from tqdm import tqdm

## Read the Train Data

In [5]:
data = pd.read_csv('/content/Train.csv')
data.index = data.date_time
data = data.drop(['date_time'],axis=1)
data.head()

,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume
date_time,,,,,,,,,,,,,,
2012-10-02 09:00:00,NaN,121,89,2,329,1,1,288.28,0.0,0,40,Clouds,scattered clouds,5545.0
2012-10-02 10:00:00,NaN,178,67,3,330,1,1,289.36,0.0,0,75,Clouds,broken clouds,4516.0
2012-10-02 11:00:00,NaN,113,66,3,329,2,2,289.58,0.0,0,90,Clouds,overcast clouds,4767.0
2012-10-02 12:00:00,NaN,20,66,3,329,5,5,290.13,0.0,0,90,Clouds,overcast clouds,5026.0
2012-10-02 13:00:00,NaN,281,65,3,329,7,7,291.14,0.0,0,75,Clouds,broken clouds,4918.0


## Data Preprocessing
For handling categorical variables is_holiday, weather_type, weather_description, we perform one-hot encoding

In [6]:
def pre_process(data):
    data['holiday'] = 0
    for i in tqdm(range(len(data))):
        if(data.iloc[i]['is_holiday'] != "None"):
            data.iloc[i]['holiday'] = 1
    weather_type = pd.get_dummies(data['weather_type'],prefix="weather_type")
    weather_desc = pd.get_dummies(data['weather_description'],prefix="weather_desc")
    data = data.drop(['weather_type','weather_description','is_holiday'],axis=1)
    data = pd.concat([data,weather_type,weather_desc],axis=1)
    data.head()
    return(data)

In [7]:
data = pre_process(data)

  0%|          | 0/13313 [00:00<?, ?it/s]<ipython-input-6-5fe0e72183f9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[i]['holiday'] = 1
100%|██████████| 13313/13313 [00:02<00:00, 4508.42it/s]


## Train Data Assignment

In [8]:
train = data

## Specify endogenous and exogenous variables in the data

In [9]:
# Variables
exog_data = train.drop(['traffic_volume'],axis=1)
exog = sm.add_constant(exog_data)
endog = train[[u'traffic_volume']]

print(endog)
print(exog)
# nobs = endog.shape[0]

                     traffic_volume
date_time                          
2012-10-02 09:00:00          5545.0
2012-10-02 10:00:00          4516.0
2012-10-02 11:00:00          4767.0
2012-10-02 12:00:00          5026.0
2012-10-02 13:00:00          4918.0
...                             ...
2014-03-31 00:00:00           582.0
2014-03-31 01:00:00           365.0
2014-03-31 02:00:00           359.0
2014-03-31 03:00:00           364.0
2014-03-31 04:00:00             NaN

[13313 rows x 1 columns]
                     const  air_pollution_index  humidity  wind_speed  \
date_time                                                               
2012-10-02 09:00:00    1.0                  121        89           2   
2012-10-02 10:00:00    1.0                  178        67           3   
2012-10-02 11:00:00    1.0                  113        66           3   
2012-10-02 12:00:00    1.0                   20        66           3   
2012-10-02 13:00:00    1.0                  281        65           

In [16]:
# Convert the data to NumPy arrays
print(endog.dtype)
print(exog.dtype)
if endog.dtype != np.float64:
    endog = endog.astype(np.float64)
if exog.dtype != np.float64:
    exog = exog.astype(np.float64)

float64
object


## Train the Model (Slow Cell)

In [17]:
# Fit the model
mod = sm.tsa.statespace.SARIMAX(endog, exog=exog, order=(1,0,1))
fit_res = mod.fit(disp=False)
print(fit_res.summary())

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                13313
Model:               SARIMAX(1, 0, 1)   Log Likelihood             -108201.617
Date:                Mon, 15 Apr 2024   AIC                         216523.233
Time:                        09:20:26   BIC                         216973.023
Sample:                             0   HQIC                        216673.344
                              - 13313                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2403.1344    120.017    -20.023      0.000   -2638.364   -2167.905
x1            -0.0979      0.053     -1.836      0.066      -0.202       0.007
x2             0.8880      0.883      1.006      0.3

## Read in Test Set

In [18]:
test_set = pd.read_csv("/content/Test.csv")
test_set.index = test_set.date_time
test_set = test_set.drop(['date_time'],axis=1)
test_set.head()

,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description
date_time,,,,,,,,,,,,,
2017-05-18 00:00:00,NaN,73,63,1,27,4,4,285.15,0.0,0,90,Rain,moderate rain
2017-05-18 00:00:00,NaN,251,63,1,27,4,4,285.15,0.0,0,90,Mist,mist
2017-05-18 00:00:00,NaN,75,56,1,0,1,1,285.15,0.0,0,90,Drizzle,light intensity drizzle
2017-05-18 01:00:00,NaN,98,56,1,351,2,2,284.79,0.0,0,90,Rain,heavy intensity rain
2017-05-18 01:00:00,NaN,283,56,1,351,1,1,284.79,0.0,0,90,Mist,mist


In [19]:
test_set = pre_process(test_set)

  0%|          | 0/14454 [00:00<?, ?it/s]<ipython-input-6-5fe0e72183f9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[i]['holiday'] = 1
100%|██████████| 14454/14454 [00:03<00:00, 4386.95it/s]


In [20]:
test_set.head()

,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,...,weather_desc_shower drizzle,weather_desc_sky is clear,weather_desc_sleet,weather_desc_smoke,weather_desc_snow,weather_desc_thunderstorm,weather_desc_thunderstorm with heavy rain,weather_desc_thunderstorm with light drizzle,weather_desc_thunderstorm with light rain,weather_desc_thunderstorm with rain
date_time,,,,,,,,,,,,,,,,,,,,,
2017-05-18 00:00:00,73,63,1,27,4,4,285.15,0.0,0,90,...,False,False,False,False,False,False,False,False,False,False
2017-05-18 00:00:00,251,63,1,27,4,4,285.15,0.0,0,90,...,False,False,False,False,False,False,False,False,False,False
2017-05-18 00:00:00,75,56,1,0,1,1,285.15,0.0,0,90,...,False,False,False,False,False,False,False,False,False,False
2017-05-18 01:00:00,98,56,1,351,2,2,284.79,0.0,0,90,...,False,False,False,False,False,False,False,False,False,False
2017-05-18 01:00:00,283,56,1,351,1,1,284.79,0.0,0,90,...,False,False,False,False,False,False,False,False,False,False


## Handling columns that aren't present in the test set, but are in the train set

In [21]:
for i in train.columns:
    if i not in test_set.columns:
        test_set[i] = 0
test_set.tail()

,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,...,weather_desc_thunderstorm with rain,traffic_volume,weather_type_Clo,weather_type_Squall,weather_desc_SQUALLS,weather_desc_freezing rain,weather_desc_light rain and snow,weather_desc_shower snow,weather_desc_thunderstorm with drizzle,weather_desc_very heavy rain
date_time,,,,,,,,,,,,,,,,,,,,,
2018-09-30 19:00:00,176,21,4,345,4,4,283.45,0.0,0,75,...,False,0,0,0,0,0,0,0,0,0
2018-09-30 20:00:00,214,95,8,280,6,6,282.76,0.0,0,90,...,False,0,0,0,0,0,0,0,0,0
2018-09-30 21:00:00,173,63,4,238,1,1,282.73,0.0,0,90,...,False,0,0,0,0,0,0,0,0,0
2018-09-30 22:00:00,21,57,8,268,7,7,282.09,0.0,0,90,...,False,0,0,0,0,0,0,0,0,0
2018-09-30 23:00:00,116,70,8,328,6,6,282.12,0.0,0,90,...,False,0,0,0,0,0,0,0,0,0


## Forecasting the traffic_volume for the given test set

In [26]:
# last_train = train.iloc[len(train)-1].name
first_predict = test_set.iloc[0].name
# print(last_train,first_predict)
# import datetime as dt

# start_dt = dt.datetime.strptime(last_train, '%Y-%m-%d %H:%M:%S')
# predict_dt = dt.datetime.strptime(first_predict, '%Y-%m-%d %H:%M:%S')
# diff = (predict_dt - start_dt)
# days, seconds = diff.days, diff.seconds
# hours = days * 24 + seconds // 3600
# print(hours)

exog1 = (sm.add_constant(test_set).loc[first_predict:])
exog1 = exog1.drop(['traffic_volume'],axis=1)

# print(pd.concat([exog,exog1]))
# predict = fit_res.predict(start=hours,end=hours,exog=exog1)
# print(predict)

print(exog1)

# Impute missing values with the mean
exog1 = exog1.fillna(exog1.mean())

# Convert data type to float
exog1 = exog1.astype(float)

# Reshape exog1 to match the model
exog1 = exog1.iloc[:, :-5]

print(exog1.shape)
forecast = fit_res.forecast(steps = len(test_set),exog = exog1)
print(forecast,len(forecast),len(test_set))

                     const  air_pollution_index  humidity  wind_speed  \
date_time                                                               
2017-05-18 00:00:00    1.0                   73        63           1   
2017-05-18 00:00:00    1.0                  251        63           1   
2017-05-18 00:00:00    1.0                   75        56           1   
2017-05-18 01:00:00    1.0                   98        56           1   
2017-05-18 01:00:00    1.0                  283        56           1   
...                    ...                  ...       ...         ...   
2018-09-30 19:00:00    1.0                  176        21           4   
2018-09-30 20:00:00    1.0                  214        95           8   
2018-09-30 21:00:00    1.0                  173        63           4   
2018-09-30 22:00:00    1.0                   21        57           8   
2018-09-30 23:00:00    1.0                  116        70           8   

                     wind_direction  visibility_in

In [27]:
result_data = pd.DataFrame(index=test_set.index, columns=['date_time','traffic_volume'])
result_data.head()

,date_time,traffic_volume
date_time,,
2017-05-18 00:00:00,NaN,NaN
2017-05-18 00:00:00,NaN,NaN
2017-05-18 00:00:00,NaN,NaN
2017-05-18 01:00:00,NaN,NaN
2017-05-18 01:00:00,NaN,NaN


In [28]:
chk = 0
for i in tqdm(forecast):
    result_data.iloc[chk]["date_time"] = test_set.iloc[chk].name
    result_data.iloc[chk]["traffic_volume"] = i
    chk+=1
result_data.head()

100%|██████████| 14454/14454 [00:02<00:00, 4951.78it/s]


,date_time,traffic_volume
date_time,,
2017-05-18 00:00:00,2017-05-18 00:00:00,2916.441674
2017-05-18 00:00:00,2017-05-18 00:00:00,1801.05229
2017-05-18 00:00:00,2017-05-18 00:00:00,2577.810341
2017-05-18 01:00:00,2017-05-18 01:00:00,2602.146858
2017-05-18 01:00:00,2017-05-18 01:00:00,2545.516623


In [30]:
result_data.to_csv('results.csv', header=['date_time','traffic_volume'], index=False)